In [1]:
# !unzip archive.zip

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
from tqdm import tqdm
import numpy as np

/home/zahoor-ai-developer/Documents/fall-2024/CV-LAB/i202384_A3/a3_task_updated/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import cv2
# import time

# # Path to the video file
# video_path = "HMDB_dataset/jump/testing_1.avi"

# # Open the video file
# cap = cv2.VideoCapture(video_path)

# if not cap.isOpened():
#     print("Error: Could not open video.")
#     exit()

# # Calculate FPS using the video file's metadata
# fps_metadata = cap.get(cv2.CAP_PROP_FPS)
# print(f"FPS from metadata: {fps_metadata}")

# # Alternatively, calculate FPS dynamically by measuring the time to process frames
# frame_count = 0
# start_time = time.time()

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     frame_count += 1

# end_time = time.time()

# print(frame_count)
# # Calculate FPS dynamically
# duration = end_time - start_time
# fps_dynamic = frame_count / duration if duration > 0 else 0

# print(f"FPS calculated dynamically: {fps_dynamic:.2f}")

# # Release the video capture object
# cap.release()


In [3]:
# import os
# import cv2
# from tqdm import tqdm

# def extract_frames_from_videos(
#     dataset_path,
#     output_path,
#     stride=10,
#     video_extensions=(".mp4", ".avi", ".mov", ".mkv")
# ):
#     """
#     Extracts every nth frame from videos in the dataset and organizes them into class-specific folders.

#     Parameters:
#     - dataset_path (str): Path to the HMDB_dataset directory.
#     - output_path (str): Path where the Frames_dataset will be saved.
#     - stride (int): Interval of frames to extract (e.g., every 10th frame).
#     - video_extensions (tuple): Accepted video file extensions.
#     """
    
#     # Ensure the output directory exists
#     os.makedirs(output_path, exist_ok=True)
    
#     # List all class directories in the dataset
#     classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    
#     if not classes:
#         print(f"No class directories found in {dataset_path}. Please check the dataset path.")
#         return
    
#     for class_name in classes:
#         class_input_path = os.path.join(dataset_path, class_name)
#         class_output_path = os.path.join(output_path, class_name)
        
#         # Create the class output directory if it doesn't exist
#         os.makedirs(class_output_path, exist_ok=True)
        
#         # List all video files in the class directory
#         videos = [
#             f for f in os.listdir(class_input_path)
#             if os.path.isfile(os.path.join(class_input_path, f)) and f.lower().endswith(video_extensions)
#         ]
        
#         if not videos:
#             print(f"No video files found in {class_input_path}. Skipping this class.")
#             continue
        
#         print(f"\nProcessing Class: {class_name} | Total Videos: {len(videos)}")
        
#         for video_file in tqdm(videos, desc=f"Extracting from {class_name}", unit="video"):
#             video_path = os.path.join(class_input_path, video_file)
#             video_name, _ = os.path.splitext(video_file)
            
#             # Initialize video capture
#             cap = cv2.VideoCapture(video_path)
            
#             if not cap.isOpened():
#                 print(f"Error: Could not open video file {video_path}. Skipping.")
#                 continue
            
#             frame_count = 0
#             saved_frames = 0
            
#             while True:
#                 ret, frame = cap.read()
#                 if not ret:
#                     break
                
#                 if frame_count % stride == 0:
#                     frame_filename = f"{video_name}_frame{frame_count:05d}.jpg"
#                     frame_path = os.path.join(class_output_path, frame_filename)
                    
#                     # Save the frame as JPEG
#                     success = cv2.imwrite(frame_path, frame)
#                     if success:
#                         saved_frames += 1
#                     else:
#                         print(f"Warning: Failed to save frame {frame_filename}.")
                
#                 frame_count += 1
            
#             cap.release()
#             # Optional: Print summary for each video
#             # print(f"Extracted {saved_frames} frames from {video_file}")

#     print("\nFrame extraction completed successfully.")

# if __name__ == "__main__":
#     # Define input and output paths
#     dataset_path = "HMDB_dataset"        # Replace with your actual dataset path
#     output_path = "Frames_dataset"       # Replace with your desired output path
    
#     # Set the stride (extract every 10th frame)
#     stride = 10
    
#     # Start frame extraction
#     extract_frames_from_videos(
#         dataset_path=dataset_path,
#         output_path=output_path,
#         stride=stride
#     )


In [4]:
# import os
# import cv2
# import random
# import shutil
# from tqdm import tqdm

# def split_frames_into_train_test(
#     input_dir,
#     output_dir,
#     train_ratio=0.8,
#     resize_dim=(224, 224),
#     seed=42,
#     image_extensions=(".jpg", ".jpeg", ".png", ".bmp", ".tiff")
# ):
#     """
#     Splits frames into training and testing sets and resizes them.

#     Parameters:
#     - input_dir (str): Path to the original frames directory.
#     - output_dir (str): Path where the split and resized frames will be saved.
#     - train_ratio (float): Proportion of frames to include in the training set.
#     - resize_dim (tuple): Desired size for the frames (width, height).
#     - seed (int): Seed for random operations to ensure reproducibility.
#     - image_extensions (tuple): Accepted image file extensions.
#     """

#     # Set the random seed for reproducibility
#     random.seed(seed)

#     # Define training and testing directories
#     train_dir = os.path.join(output_dir, "training")
#     test_dir = os.path.join(output_dir, "testing")

#     # Create the training and testing directories if they don't exist
#     os.makedirs(train_dir, exist_ok=True)
#     os.makedirs(test_dir, exist_ok=True)

#     # List all class directories in the input directory
#     classes = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

#     if not classes:
#         print(f"No class directories found in {input_dir}. Please check the input directory.")
#         return

#     for class_name in classes:
#         print(f"\nProcessing Class: {class_name}")

#         # Define paths for the current class
#         class_input_path = os.path.join(input_dir, class_name)
#         class_train_path = os.path.join(train_dir, class_name)
#         class_test_path = os.path.join(test_dir, class_name)

#         # Create class directories in training and testing folders
#         os.makedirs(class_train_path, exist_ok=True)
#         os.makedirs(class_test_path, exist_ok=True)

#         # List all image files in the class directory
#         frames = [
#             f for f in os.listdir(class_input_path)
#             if os.path.isfile(os.path.join(class_input_path, f)) and f.lower().endswith(image_extensions)
#         ]

#         if not frames:
#             print(f"No image files found in {class_input_path}. Skipping this class.")
#             continue

#         # Shuffle frames to ensure random splitting
#         random.shuffle(frames)

#         # Calculate split index
#         split_idx = int(len(frames) * train_ratio)

#         # Split frames into training and testing sets
#         train_frames = frames[:split_idx]
#         test_frames = frames[split_idx:]

#         print(f"Total Frames: {len(frames)} | Training: {len(train_frames)} | Testing: {len(test_frames)}")

#         # Process Training Frames
#         print(f"Resizing and copying Training frames for class '{class_name}'...")
#         for frame in tqdm(train_frames, desc=f"Training | {class_name}", unit="frame"):
#             src_path = os.path.join(class_input_path, frame)
#             dst_path = os.path.join(class_train_path, frame)

#             # Read the image
#             image = cv2.imread(src_path)
#             if image is None:
#                 print(f"Warning: Failed to read {src_path}. Skipping.")
#                 continue

#             # Resize the image
#             resized_image = cv2.resize(image, resize_dim)

#             # Save the resized image
#             success = cv2.imwrite(dst_path, resized_image)
#             if not success:
#                 print(f"Warning: Failed to write {dst_path}.")

#         # Process Testing Frames
#         print(f"Resizing and copying Testing frames for class '{class_name}'...")
#         for frame in tqdm(test_frames, desc=f"Testing | {class_name}", unit="frame"):
#             src_path = os.path.join(class_input_path, frame)
#             dst_path = os.path.join(class_test_path, frame)

#             # Read the image
#             image = cv2.imread(src_path)
#             if image is None:
#                 print(f"Warning: Failed to read {src_path}. Skipping.")
#                 continue

#             # Resize the image
#             resized_image = cv2.resize(image, resize_dim)

#             # Save the resized image
#             success = cv2.imwrite(dst_path, resized_image)
#             if not success:
#                 print(f"Warning: Failed to write {dst_path}.")

#     print("\nFrame splitting and resizing completed successfully.")

# if __name__ == "__main__":
#     # Define input and output paths
#     input_frames_dir = "Frames_dataset"    # Replace with your actual frames directory
#     output_frames_dir = "Frames_dataset_split"      # Replace with your desired output directory

#     # Parameters
#     training_ratio = 0.8                            # 80% training, 20% testing
#     resize_dimensions = (224, 224)                   # Resize frames to 224x224 pixels
#     random_seed = 42                                 # Seed for reproducibility

#     # Start the splitting and resizing process
#     split_frames_into_train_test(
#         input_dir=input_frames_dir,
#         output_dir=output_frames_dir,
#         train_ratio=training_ratio,
#         resize_dim=resize_dimensions,
#         seed=random_seed
#     )


In [5]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version (PyTorch):", torch.version.cuda)
print("cuDNN Version:", torch.backends.cudnn.version())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


PyTorch Version: 2.5.1+cu124
CUDA Available: True
CUDA Version (PyTorch): 12.4
cuDNN Version: 90100
GPU Name: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [6]:
# img = cv2.imread("Frames_dataset_split/testing/jump/testing_2_frame00030.jpg")
# img.shape

In [7]:
print(torch.__version__)

2.5.1+cu124


In [8]:
# ==========================
# Configuration and Setup
# ==========================

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths to the training and testing directories
train_dir = "Frames_dataset_split/training"  # Replace with your training frames directory
test_dir = "Frames_dataset_split/testing"    # Replace with your testing frames directory

# Hyperparameters
num_classes = 51          # Number of target classes
learning_rate = 1e-4     # Learning rate for optimizer
num_epochs = 5          # Number of training epochs
batch_size = 32          # Batch size for DataLoader
stride = 10              # Stride used during frame extraction (if applicable)

# Initialize the feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')


Using device: cuda


/home/zahoor-ai-developer/Documents/fall-2024/CV-LAB/i202384_A3/a3_task_updated/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [9]:
# Define image transformations using the feature extractor
def get_transforms():
    return transforms.Compose([
        transforms.Resize((224, 224)),               # Resize images to 224x224
        transforms.ToTensor(),                       # Convert PIL Image to Tensor
        transforms.Normalize(mean=feature_extractor.image_mean,
                             std=feature_extractor.image_std),  # Normalize based on feature extractor
    ])

# Create datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=get_transforms())
test_dataset = datasets.ImageFolder(root=test_dir, transform=get_transforms())

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Class names
class_names = train_dataset.classes
print(f"Classes: {class_names}")

Classes: ['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


In [10]:
# ==========================
# Model Setup
# ==========================

# Load the pretrained ViT model with num_labels=51
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=num_classes,              # Set to 51
    ignore_mismatched_sizes=True        # Ignore size mismatch for the classifier head
)

# Freeze all model parameters
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the classifier head parameters
for param in model.classifier.parameters():
    param.requires_grad = True

# Move the model to the configured device
model.to(device)

# Print model architecture (optional)
print(model)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([51]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([51, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [11]:
# ==========================
# Loss Function and Optimizer
# ==========================

criterion = nn.CrossEntropyLoss()

# Only parameters of the classifier head are being optimized
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)


In [12]:
# ==========================
# Training and Evaluation Functions
# ==========================

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(dataloader, desc="Training", leave=False):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

In [13]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating", leave=False):
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            
            # Statistics
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

In [14]:
# ==========================
# Main Training Loop
# ==========================

best_test_acc = 0.0
best_model_path = "best_vit_model.pth"

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 30)
    
    # Train the model
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    print(f"Training   Loss: {train_loss:.4f} | Accuracy: {train_acc:.2f}%")
    
    # Evaluate the model
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    print(f"Testing    Loss: {test_loss:.4f} | Accuracy: {test_acc:.2f}%")
    
    # Save the model if it has the best accuracy so far
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"Best model updated with accuracy: {best_test_acc:.2f}%")

print(f"\nTraining completed. Best Test Accuracy: {best_test_acc:.2f}%")
print(f"Best model saved to {best_model_path}")


Epoch 1/5
------------------------------


Training:   0%|          | 0/1647 [00:00<?, ?it/s]

Training   Loss: 1.8250 | Accuracy: 56.58%


Testing    Loss: 1.2402 | Accuracy: 67.98%
Best model updated with accuracy: 67.98%

Epoch 2/5
------------------------------


Training   Loss: 1.0686 | Accuracy: 72.36%


...

Testing    Loss: 0.8735 | Accuracy: 76.78%
Best model updated with accuracy: 76.78%

Epoch 4/5
------------------------------


Training   Loss: 0.7744 | Accuracy: 79.59%


Testing    Loss: 0.7997 | Accuracy: 78.94%
Best model updated with accuracy: 78.94%

Epoch 5/5
------------------------------


Training   Loss: 0.7017 | Accuracy: 81.34%


Testing    Loss: 0.7473 | Accuracy: 79.89%
Best model updated with accuracy: 79.89%

Training completed. Best Test Accuracy: 79.89%
Best model saved to best_vit_model.pth


In [ ]:
# ==========================
# Load the Best Model for Evaluation
# ==========================

# To load the best model for future use:
# model.load_state_dict(torch.load(best_model_path))
# model.to(device)
# model.eval()

# ==========================
# Example Inference
# ==========================

def predict(model, image_path, feature_extractor, device, class_names):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    inputs = feature_extractor(images=image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        confidence, predicted_class = torch.max(probabilities, dim=1)
    
    print(f"Predicted Class: {class_names[predicted_class.item()]} | Confidence: {confidence.item():.4f}")

# Example usage:
# predict(model, "path_to_sample_image.jpg", feature_extractor, device, class_names)

